In [180]:
import pandas as pd
from globals import YELP_DIR
import json
import os

YELP_DIR = "/Volumes/Forster Neu/Masterarbeit Data/yelp_dataset/"
ORIGINAL_DATA_FOLDER = "/Users/andreaforster/dev/thesis/CAPRI/Data/Yelp/"
dataset = "yelp"
DATASET_DIR = f"/Volumes/Forster Neu/Masterarbeit Data/{dataset}_dataset/"

This Yelp dataset was used in the following script: https://www.yelp.com/dataset

In [48]:
business_df = pd.read_json(YELP_DIR + "yelp_academic_dataset_business.json", lines=True)

In [49]:
timestamp_df = pd.read_json(YELP_DIR + "yelp_academic_dataset_checkin.json", lines=True)

In [50]:
def open_big_json(file_path):
    data = []

    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON data and append to the list
            data.append(json.loads(line))

    # Create a DataFrame from the list of records
    df = pd.DataFrame(data)

    # Print the first few rows of the DataFram

    return df

In [198]:
user_df = open_big_json(YELP_DIR + "yelp_academic_dataset_user.json")
review_df = open_big_json(YELP_DIR + "yelp_academic_dataset_review.json")

: 

In [ ]:
checkin_df_filtered = review_df.drop(["text", "cool", "stars", "useful", "funny", "review_id"], axis=1)

In [ ]:
def filter_df(df, min_reviews=10):
    while True:
        # Filter businesses with at least min_reviews reviews
        business_counts = df["business_id"].value_counts()
        business_mask = df['business_id'].map(business_counts) >= min_reviews
        df_filtered = df.loc[business_mask]

        # Filter users with at least min_reviews reviews
        user_counts = df_filtered['user_id'].value_counts()
        user_mask = df_filtered['user_id'].map(user_counts) >= min_reviews
        df_filtered = df_filtered.loc[user_mask]

        # If the size of the filtered DataFrame didn't change, break the loop
        if df_filtered.shape[0] == df.shape[0]:
            break

        # Update the DataFrame for the next iteration
        df = df_filtered

    return df_filtered

In [ ]:
checkin_df_filtered = filter_df(checkin_df_filtered)

In [ ]:
# Step 1: Calculate the value counts of `business_id`
value_counts = checkin_df_filtered['business_id'].value_counts().reset_index()
value_counts.columns = ['business_id', 'count']

# Step 2: Normalize the counts y dividing by the maximum value count
max_count = value_counts['count'].max()
value_counts['business_popularity'] = value_counts['count'] / max_count

# Step 3: Merge the normalized counts back into the original DataFrame
checkin_df_filtered = checkin_df_filtered.merge(value_counts[['business_id', 'business_popularity']], on='business_id', how='left')


In [ ]:
checkin_df_filtered


,user_id,business_id,date,business_popularity
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,2018-07-07 22:09:11,0.029095
1,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,2014-02-05 20:30:30,0.003592
2,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,2017-01-14 20:54:15,0.007184
3,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,2009-10-14 19:57:14,0.077586
4,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,2011-10-27 17:12:05,0.093750
...,...,...,...,...
2762093,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,2015-01-23 23:52:03,0.053520
2762094,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,2014-12-17 21:45:20,0.004310
2762095,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,2019-12-30 03:56:30,0.006106
2762096,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,2022-01-19 18:59:27,0.015445


In [ ]:
# # filtering out users and businesses with less than 10 reviews

# business_counts = review_df["business_id"].value_counts()
# mask = review_df['business_id'].map(business_counts) >= 10
# review_df_filtered = review_df.loc[mask]

# counts = review_df_filtered['user_id'].value_counts()
# mask = review_df_filtered['user_id'].map(counts) >= 10
# review_df_filtered = review_df_filtered.loc[mask]


# # dropping text column
# review_df_filtered = review_df_filtered.drop("text", axis=1)
# # turning reviews (stars) into implicit feedback (binary)
# #review_df_filtered["stars_bin"] = review_df_filtered.apply(review_binary, axis=1)

# # getting the ids of the remaining users and businesses
# users_with_more_than_10_reviews = review_df_filtered['user_id'].unique()
# businesses_with_more_than_10_reviews = review_df_filtered['business_id'].unique()

# # filtering out the other users and businesses from the original dataframes
# user_df_filtered = user_df[user_df['user_id'].isin(users_with_more_than_10_reviews)]
# business_df_filtered = business_df[business_df['business_id'].isin(businesses_with_more_than_10_reviews)]


# # getting the highest number of reviews for a business to normalize the popularity of businesses
# max_num_reviews = business_df_filtered["review_count"].max()
# business_df_filtered["popularity"] = business_df_filtered["review_count"] / max_num_reviews 

In [ ]:
def user_popularity_calculator(checkin_df_filtered, user_df, business_df):
    # Filter out instances with the specified business_id
    try:
        checkin_df_filtered = checkin_df_filtered[checkin_df_filtered['business_id'] != "00000000000000000000000000000000"]
    except Exception as e:
        print("No such field found to filter out")
    # Calculate average popularity per user
    average_popularity_per_user = checkin_df_filtered.groupby('user_id')['business_popularity'].mean().reset_index()
    average_popularity_per_user.columns = ['user_id', 'average_popularity']

    average_popularity_per_user = average_popularity_per_user.sort_values(by="average_popularity", ascending=False)

    
    # Sort by average popularity
    

    # Get top 1000 users
    high_pop_user_df_sample = average_popularity_per_user.head(1500)
    
    # Get the middle 1000 users around the median
    median_index = len(average_popularity_per_user) // 2
    start_med_index = max(median_index - 750, 0)
    end_med_index = min(median_index + 750, len(average_popularity_per_user))
    medium_pop_user_df_sample = average_popularity_per_user.iloc[start_med_index:end_med_index]
    
    # Get the lowest 1000 users
    low_pop_user_df_sample = average_popularity_per_user.tail(1500)

    unique_users = list(set(high_pop_user_df_sample["user_id"].tolist() + medium_pop_user_df_sample["user_id"].tolist() + low_pop_user_df_sample["user_id"].tolist()))

    checkin_df_sample = checkin_df_filtered[checkin_df_filtered["user_id"].isin(unique_users)]
    #checkin_df_sample = filter_df(checkin_df_sample, min_reviews=10)
    
    unique_businesses = list(checkin_df_sample["business_id"].unique())

    # business_counts = checkin_df_sample["business_id"].value_counts()
    # business_mask = checkin_df_sample['business_id'].map(business_counts) >=10
    # checkin_df_sample = checkin_df_sample.loc[business_mask]

    
    return checkin_df_sample, high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, business_df_sample

In [ ]:
checkin_df_sample, high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample, user_df_sample, business_df_sample = user_popularity_calculator(checkin_df_filtered, user_df, business_df)

In [ ]:
len(low_pop_user_df_sample)

1500

In [ ]:
len(checkin_df_sample["business_id"].unique())

1622

In [ ]:
def data_saver(df, filename, framework):
    if not os.path.exists(YELP_DIR + "processed_data_" + framework):
        os.makedirs(YELP_DIR + "processed_data_" + framework)
    
    df.to_csv(YELP_DIR + "processed_data_" + framework + "/" + filename + ".csv")
    print("Data saved as " + framework + filename + ".csv")
    


In [ ]:
# first of all saving data for cornac
data_saver(checkin_df_sample, "user_events", "cornac")
data_saver(business_df_sample, "businesses", "cornac")
data_saver(high_pop_user_df_sample, "high_pop_user_sample", "cornac")
data_saver(medium_pop_user_df_sample, "medium_pop_user_sample", "cornac")
data_saver(low_pop_user_df_sample, "low_pop_user_sample", "cornac")


Data saved as cornacuser_events.csv
Data saved as cornacbusinesses.csv
Data saved as cornachigh_pop_user_sample.csv
Data saved as cornacmedium_pop_user_sample.csv
Data saved as cornaclow_pop_user_sample.csv


In [ ]:
def data_saver_recbole(df, framework, suffix):
    
    if not os.path.exists(YELP_DIR + "processed_data_" + framework):
        os.makedirs(YELP_DIR + "processed_data_" + framework)

    df.to_csv(f"{YELP_DIR}processed_data_{framework}/yelp_sample.{suffix}", sep="\t", index=False)

In [ ]:
checkin_df_sample['review_id'] = range(1, len(checkin_df_sample) + 1)
# Step 1: Group by user_id and business_id and count check-ins
checkin_df_sample['checkin_count'] = checkin_df_sample.groupby(['user_id', 'business_id'])['business_id'].transform('count')


In [ ]:
checkin_df_sample

,user_id,business_id,date,business_popularity,review_id,checkin_count
44,mEOMAeEonZoUx2nPM3v6fg,f-WhNOSwN1aB4nRFekf01g,2015-03-19 00:30:09,0.011853,1,1
72,7Ie0VmQtnGYUVq2YW4dTVw,xruWHK8Z5N0JWyQubLHjgA,2011-02-14 16:56:15,0.019037,2,1
261,wo1qycEKVbUYhezpmialJA,N-MVwAcmsQDZCxbZjZhPDA,2012-09-11 02:37:42,0.009339,3,1
266,758g6NGLp9deCbvowz62Ww,z6SVTb9eFIcWVpKXIfFEvQ,2008-12-03 20:05:18,0.016164,4,1
272,TmdJhSSDABdUwuci8HhbBw,cXSyVvOr9YRN9diDkaWs0Q,2008-05-06 18:40:32,0.333693,5,1
...,...,...,...,...,...,...
2761993,rBeKFxV0PJ-wXMjaEa8G5A,1hZCHWFUixqG77pEqpubYQ,2015-11-27 01:49:31,0.017960,102200,1
2762011,wlUfN4_5MPnuA3xFAjV2_g,rgeuy1qbw6Z8B6CSVANHIA,2013-06-21 20:58:26,0.307830,102201,1
2762017,evRwCG9m-lwfL5TeCwrAhg,qQO7ErS_RAN4Vs1uX0L55Q,2010-08-29 13:46:44,0.460489,102202,1
2762032,KhZhgm_v96Q7XBpbZv9xSA,s-d9PNQJELDmTtKQCnFSvA,2021-12-02 13:27:19,0.004310,102203,9


In [ ]:
business_sample = business_df_sample[["business_id", "latitude", "longitude"]]
business_sample = business_sample.rename(columns={"latitude": "lat", "longitude": "lon"})

In [ ]:
business_sample

,business_id,lat,lon
3,MTSW4McQd7CbVtyjqoe9mw,39.955505,-75.155564
6,n_0UpQx1hsNbnPUSlodU8w,38.627695,-90.340465
15,MUTTqe8uqyMdBl186RmNeA,39.953949,-75.143226
19,ROeacJQwBeh05Rqg7F6TCg,39.943223,-75.162568
20,WKMJwqnfZKsAae75RMP6jA,53.546045,-113.499169
...,...,...,...
150322,2MAQeAqmD8enCT2ZYqUgIQ,36.163875,-86.776311
150323,w_4xUt-1AyY2ZwKtnjW0Xg,43.616590,-116.202383
150327,cM6V90ExQD6KMSU3rRB5ZA,43.615401,-116.284689
150328,1jx1sfgjgVg0nM6n3p0xWA,32.409552,-110.943073


In [ ]:
user_df_sample = checkin_df_sample.groupby('user_id').size().reset_index(name='review_counts:float')
checkin_df_sample = checkin_df_sample[["review_id","user_id","business_id","date", "checkin_count"]]


In [ ]:
user_df_sample

,user_id,review_counts:float
0,--Hno7jSp4AeOC1vq2Ohew,14
1,--_H9j6ggxvqhh9nPofZwg,41
2,-0LGLx8LP5dq3zcGO4Bebw,26
3,-0YrXUvXz8112yHap35V2g,52
4,-0fzr1jyxY2VzRdirdSR_A,13
...,...,...
4495,zxMo7emoqvWavJRtJF_9rw,46
4496,zyab2qsMarU5bGjhRL7syw,10
4497,zyoR-051xmIhp0BdS_sTsA,12
4498,zyvxtbh5eJ86bVgk52Yflg,296


In [ ]:
checkin_df_sample

,review_id,user_id,business_id,date,checkin_count
44,1,mEOMAeEonZoUx2nPM3v6fg,f-WhNOSwN1aB4nRFekf01g,2015-03-19 00:30:09,1
72,2,7Ie0VmQtnGYUVq2YW4dTVw,xruWHK8Z5N0JWyQubLHjgA,2011-02-14 16:56:15,1
261,3,wo1qycEKVbUYhezpmialJA,N-MVwAcmsQDZCxbZjZhPDA,2012-09-11 02:37:42,1
266,4,758g6NGLp9deCbvowz62Ww,z6SVTb9eFIcWVpKXIfFEvQ,2008-12-03 20:05:18,1
272,5,TmdJhSSDABdUwuci8HhbBw,cXSyVvOr9YRN9diDkaWs0Q,2008-05-06 18:40:32,1
...,...,...,...,...,...
2761993,102200,rBeKFxV0PJ-wXMjaEa8G5A,1hZCHWFUixqG77pEqpubYQ,2015-11-27 01:49:31,1
2762011,102201,wlUfN4_5MPnuA3xFAjV2_g,rgeuy1qbw6Z8B6CSVANHIA,2013-06-21 20:58:26,1
2762017,102202,evRwCG9m-lwfL5TeCwrAhg,qQO7ErS_RAN4Vs1uX0L55Q,2010-08-29 13:46:44,1
2762032,102203,KhZhgm_v96Q7XBpbZv9xSA,s-d9PNQJELDmTtKQCnFSvA,2021-12-02 13:27:19,9


In [ ]:
def convert_to_unix_timestamp(df, column_name):
    """
    Convert a column of timestamps in a DataFrame to Unix timestamps.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamp column.
        column_name (str): The name of the column with timestamps in "%Y-%m-%d %H:%M:%S" format.

    Returns:
        pd.DataFrame: The DataFrame with an additional column for Unix timestamps.
    """
    # Convert the column to datetime objects
    df[column_name] = pd.to_datetime(df[column_name], format="mixed")
    
    # Convert datetime objects to Unix timestamps
    df[f'{column_name}'] = df[column_name].apply(lambda x: x.timestamp())
    
    return df

In [ ]:
checkin_df_sample.rename(columns={"user_id":"user_id:token", "business_id":"item_id:token", "checkin_count":"rating:float", "date":"timestamp:float", "review_id": "review_id:token"}, inplace=True)
user_df_sample.rename(columns={"user_id":"user_id:token"}, inplace=True)
business_sample.rename(columns={"business_id": "item_id:token", "lat" : "lat:float", "lon" : "lon:float"}, inplace=True)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_63008/699706694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkin_df_sample.rename(columns={"user_id":"user_id:token", "business_id":"item_id:token", "checkin_count":"rating:float", "date":"timestamp:float", "review_id": "review_id:token"}, inplace=True)


In [ ]:
checkin_df_sample = convert_to_unix_timestamp(checkin_df_sample, "timestamp:float")

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_63008/869447620.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_datetime(df[column_name], format="mixed")
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_63008/869447620.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column_name}'] = df[column_name].apply(lambda x: x.timestamp())


In [ ]:
#checkin_df_sample = checkin_df_sample.loc[checkin_df_sample["rating:float"] < 100]

checkin_df_sample.sort_values(by="rating:float", ascending=False)

checkin_df_timestamps = checkin_df_sample.copy()

checkin_df_sample = checkin_df_sample.drop_duplicates(subset=["user_id:token", "item_id:token"], keep="first")

In [ ]:
checkin_df_timestamps["user_id:token"].nunique()

4500

In [ ]:
data_saver_recbole(checkin_df_sample, "recbole", "inter")
data_saver_recbole(user_df_sample, "recbole", "user")
data_saver_recbole(business_sample, "recbole", "item")

In [ ]:
# reverting to unix timestamps and only using the necessary columns
review_df_sample = checkin_df_sample.copy()
checkins_capri_min = checkin_df_timestamps[["user_id:token", "item_id:token", "timestamp:float"]]

checkins_capri_train_test_tune = review_df_sample[["user_id:token", "item_id:token", "timestamp:float", "rating:float"]]

In [ ]:
len(checkins_capri_min["user_id:token"].unique())
len(checkins_capri_min["item_id:token"].unique())

print("Length of users: ", len(checkins_capri_min["user_id:token"].unique()))
print("Length of POIs: ", len(checkins_capri_min["item_id:token"].unique()))



Length of users:  4500
Length of POIs:  32811


In [ ]:
# FINAL 3
datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_min["user_id:token"].unique())], "num_items" : [len(checkins_capri_min["item_id:token"].unique())]})

In [ ]:
datasize_capri

,num_users,num_items
0,4500,32811


In [ ]:
user_df_sample

,user_id,average_popularity
84385,qdQH17-Me0AgtAN6RekeSQ,0.572329
81799,oyyPJCsDlyYo-xLHmw10yw,0.433585
14626,8OBgJJF-3CYIy69RLqwz3w,0.395801
87361,sXtYdtCP3Ew1gHfxJxk51w,0.386592
24885,F516eVATM3c7pLqLHNUk_Q,0.382759
...,...,...
65461,eS6HSHjlwbTNuKlygZYVVQ,0.005603
36828,Mt3yZiebPuJrQxeEK2nUGg,0.005244
33569,KhZhgm_v96Q7XBpbZv9xSA,0.005159
78638,n-OM_IDx1LRmaI7I6k-7zg,0.005127


In [ ]:
# FINAL 4
poi_coos_capri = business_sample.copy()
user_df_sample = pd.concat([high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample])



In [ ]:
user_df_sample

,user_id,average_popularity
84385,qdQH17-Me0AgtAN6RekeSQ,0.572329
81799,oyyPJCsDlyYo-xLHmw10yw,0.433585
14626,8OBgJJF-3CYIy69RLqwz3w,0.395801
87361,sXtYdtCP3Ew1gHfxJxk51w,0.386592
24885,F516eVATM3c7pLqLHNUk_Q,0.382759
...,...,...
65461,eS6HSHjlwbTNuKlygZYVVQ,0.005603
36828,Mt3yZiebPuJrQxeEK2nUGg,0.005244
33569,KhZhgm_v96Q7XBpbZv9xSA,0.005159
78638,n-OM_IDx1LRmaI7I6k-7zg,0.005127


In [ ]:
checkins_capri_train_test_tune.sort_values(by="rating:float", ascending=False)

,user_id:token,item_id:token,timestamp:float,rating:float
1348643,I6x-ZBHeCNlMnmtfUVf5lg,PUfhLM-MkjQYZw4ldnJqzQ,1.557786e+09,32
442333,I6x-ZBHeCNlMnmtfUVf5lg,cuXUuoOSZ4VbV6jvh3BC0A,1.565043e+09,22
2216471,EYC10zgoEv-d--NSE05V3A,SCXkyyKWzS7qWl2jBiWQiQ,1.420406e+09,18
2106759,F8vHZipbIVclQZEMEyOl9w,-jsmtvdoUI-GJRSklYmEuA,1.412267e+09,16
2135698,o14c28vxH5AomMc8rPIgnA,pRkw4Kfs3tsMU1bgwwB5ig,1.536143e+09,16
...,...,...,...,...
942263,UUqowo7SscAzFn927APXQw,CG1Bb2MTraJYTHCPFm72XA,1.538605e+09,1
942251,ks4yzz8mGftxdxBFJyr0og,j0ib7Ketxwsz_9F__DYWFQ,1.523502e+09,1
942207,mD4eoBM9TBoqZpZXDW8Qfg,AVBqx8FaXza6x-bb_FLOTQ,1.392435e+09,1
942177,2eFRREDckaWviORHZECqKw,kZxrObtkh34gz67C_HMl8w,1.487201e+09,1


In [ ]:
checkins_capri_min['user_id_int'], user_id_map = pd.factorize(checkins_capri_min['user_id:token'])
checkins_capri_min['business_id_int'], business_id_map = pd.factorize(checkins_capri_min['item_id:token'])

# FINAL 2.0
checkins_capri_min_int = checkins_capri_min[["user_id_int", "business_id_int", "timestamp:float"]]
# Create mapping dictionaries
user_id_mapping = {original: i for i, original in enumerate(user_id_map)}
business_id_mapping = {original: j for j, original in enumerate(business_id_map)}

poi_coos_capri["business_id_int"] = poi_coos_capri["item_id:token"].map(business_id_mapping)
# FINAL 2.0
poi_coos_capri_int = poi_coos_capri[["business_id_int", "lat:float", "lon:float"]]


# FINAL 2.0
checkins_capri_train_test_tune['user_id_int'] = checkins_capri_train_test_tune['user_id:token'].map(user_id_mapping)
checkins_capri_train_test_tune['business_id_int'] = checkins_capri_train_test_tune['item_id:token'].map(business_id_mapping)

In [ ]:
checkins_capri_train_test_tune


,user_id:token,item_id:token,timestamp:float,rating:float,user_id_int,business_id_int
44,mEOMAeEonZoUx2nPM3v6fg,f-WhNOSwN1aB4nRFekf01g,1.426725e+09,1,0,0
72,7Ie0VmQtnGYUVq2YW4dTVw,xruWHK8Z5N0JWyQubLHjgA,1.297703e+09,1,1,1
261,wo1qycEKVbUYhezpmialJA,N-MVwAcmsQDZCxbZjZhPDA,1.347331e+09,1,2,2
266,758g6NGLp9deCbvowz62Ww,z6SVTb9eFIcWVpKXIfFEvQ,1.228335e+09,1,3,3
272,TmdJhSSDABdUwuci8HhbBw,cXSyVvOr9YRN9diDkaWs0Q,1.210099e+09,1,4,4
...,...,...,...,...,...,...
2761977,5OhBh11C6dCSG5pgIvcb-A,Imj_D35_8VZ7fuVUL9Sd8w,1.442788e+09,1,2639,32138
2761993,rBeKFxV0PJ-wXMjaEa8G5A,1hZCHWFUixqG77pEqpubYQ,1.448589e+09,1,2813,29619
2762011,wlUfN4_5MPnuA3xFAjV2_g,rgeuy1qbw6Z8B6CSVANHIA,1.371848e+09,1,3420,29657
2762017,evRwCG9m-lwfL5TeCwrAhg,qQO7ErS_RAN4Vs1uX0L55Q,1.283090e+09,1,3295,29684


In [ ]:
# splitting the data into train, test, and tune
checkins_capri_train_test_tune = checkins_capri_train_test_tune.sort_values(by=["user_id_int", "timestamp:float"])
checkins_capri_train_test_tune = checkins_capri_train_test_tune[["user_id_int", "business_id_int", "rating:float"]]

# Split the data
train_list = []
val_list = []
test_list = []

for user, group in checkins_capri_train_test_tune.groupby('user_id_int'):
    n = len(group)
    train_end = int(n * 0.65)
    val_end = int(n * 0.80)
    
    train_list.append(group.iloc[:train_end])
    val_list.append(group.iloc[train_end:val_end])
    test_list.append(group.iloc[val_end:])

# Combine lists into DataFrames
train_df = pd.concat(train_list)
val_df = pd.concat(val_list)
test_df = pd.concat(test_list)



# Check the splits

# FINAL 6-8
print("Train Set:")
print(train_df.head())
print("\nValidation Set:")
print(val_df.head())
print("\nTest Set:")
print(test_df.head())


Train Set:
         user_id_int  business_id_int  rating:float
2028214            0            24680             1
599390             0             7503             1
1447869            0            17655             1
1691225            0            20607             1
2249823            0            26718             1

Validation Set:
         user_id_int  business_id_int  rating:float
1007937            0            10826             1
44                 0                0             1
2657576            0            31944             1
1995389            0            24270             1
1787655            0            21743             1

Test Set:
         user_id_int  business_id_int  rating:float
2529231            0            29817             1
2361242            0            26296             1
1033957            0            12455             1
2546262            0            30345             1
1409890            0            17019             1


In [ ]:
def datasaver_capri(df, filename):
    
    if not os.path.exists(DATASET_DIR + "processed_data_capri"):
        os.makedirs(DATASET_DIR + "processed_data_capri")
    df.to_csv(DATASET_DIR + "processed_data_capri/" + filename + ".txt", sep='\t', index=False, header=False)
    print("Data saved as " + filename + ".txt")
    

    


In [ ]:
poi_coos_capri_int.dropna(inplace=True)
poi_coos_capri_int["business_id_int"] = poi_coos_capri_int["business_id_int"].astype(int)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_63008/2553544415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_coos_capri_int.dropna(inplace=True)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_63008/2553544415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_coos_capri_int["business_id_int"] = poi_coos_capri_int["business_id_int"].astype(int)


In [ ]:
datasaver_capri(checkins_capri_min_int, "checkins")
datasaver_capri(datasize_capri, "dataSize")
datasaver_capri(poi_coos_capri_int, "poiCoos")
# datasaver_capri(social_relations_min_int, "socialRelations")
datasaver_capri(train_df, "train")
datasaver_capri(val_df, "tune")
datasaver_capri(test_df, "test")

Data saved as checkins.txt
Data saved as dataSize.txt
Data saved as poiCoos.txt
Data saved as train.txt
Data saved as tune.txt
Data saved as test.txt


*******
OLD: CATEGROY INFO 
******

In [ ]:
def unstack_categories(df, column_name):
    """
    Unstack a DataFrame column with comma-separated categories into separate rows.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column with comma-separated categories.

    Returns:
        pd.DataFrame: The DataFrame with each category in a separate row.
    """
    # Split the categories into lists
    df[column_name] = df[column_name].str.split(', ')
    
    # Explode the lists into separate rows
    df = df.explode(column_name)
    
    return df


In [ ]:
def factorize_categories(df, column_name):
    """
    Factorize categories in a DataFrame column into unique integer codes.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column with categories.

    Returns:
        pd.DataFrame: The DataFrame with factorized categories.
        pd.Index: The unique categories found in the column.
    """
    # Factorize the categories
    df[column_name + '_codes'], unique_categories = pd.factorize(df[column_name])

    return df, unique_categories

In [ ]:
# reverting to unix timestamps and only using the necessary columns
review_df_sample = convert_to_unix_timestamp(review_df_sample, "date")
#ALMOST FINAL - STARS_BIN MUST BE REMOVED FOR checkins.txt AND date_unix FOR train/test/tune.txt
checkins_capri_min = review_df_sample[["user_id", "business_id", "date_unix"]]
checkins_capri_train_test_tune = review_df_sample[["user_id", "business_id", "date_unix", "checkin_count"]]

In [ ]:
categories_capri = unstack_categories(business_df_sample, "categories")
categories_capri["category_id"], unique_categories = pd.factorize(categories_capri["categories"])


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/3621935470.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.split(', ')


In [ ]:
# filtering out categories with less than 10 POIs
category_counts = categories_capri["category_id"].value_counts()
mask = categories_capri["category_id"].map(category_counts) >= 10
categories_capri = categories_capri.loc[mask]

In [ ]:
# FINAL 2
categories_capri_min = categories_capri[["business_id", "category_id"]]

In [ ]:
len(checkins_capri_min["user_id"].unique())
len(checkins_capri_min["business_id"].unique())
len(categories_capri_min["category_id"].unique())

print("Length of users: ", len(checkins_capri_min["user_id"].unique()))
print("Length of POIs: ", len(checkins_capri_min["business_id"].unique()))
print("Length of categories_capri_min: ", len(categories_capri_min["category_id"].unique()))


Length of users:  3000
Length of POIs:  35278
Length of categories_capri_min:  645


In [ ]:
# FINAL 3
datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_min["user_id"].unique())], "num_items" : [len(checkins_capri_min["business_id"].unique())], "num_categories" : [len(categories_capri_min["category_id"].unique())]})

In [ ]:
# FINAL 4
poi_coos_capri = business_df_sample[["business_id", "latitude", "longitude"]]

In [ ]:
user_df_sample = pd.concat([high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample])

In [ ]:
social_relations_capri = unstack_categories(user_df_sample, "friends")

In [ ]:
# filtering out friends so only those that are in the subset are listed as friends
users = list(social_relations_capri["user_id"].unique())
social_relations_min = social_relations_capri.loc[social_relations_capri["friends"].isin(users)]
# FINAL 5
social_relations_min = social_relations_min[["user_id", "friends"]]

In [ ]:
checkins_capri_min['user_id_int'], user_id_map = pd.factorize(checkins_capri_min['user_id'])
checkins_capri_min['business_id_int'], business_id_map = pd.factorize(checkins_capri_min['business_id'])
# FINAL 2.0
checkins_capri_min_int = checkins_capri_min[["user_id_int", "business_id_int", "date_unix"]]
# Create mapping dictionaries
user_id_mapping = {original: i for i, original in enumerate(user_id_map)}
business_id_mapping = {original: i for i, original in enumerate(business_id_map)}
categories_capri_min['business_id_int'] = categories_capri_min['business_id'].map(business_id_mapping)
# FINAL 2.0 
categories_capri_min_int = categories_capri_min[["business_id_int", "category_id"]]
# FINAL 2.0 
categories_capri_min_int = categories_capri_min[["business_id_int", "category_id"]]
poi_coos_capri["business_id_int"] = poi_coos_capri["business_id"].map(business_id_mapping)
# FINAL 2.0
poi_coos_capri_int = poi_coos_capri[["business_id_int", "latitude", "longitude"]]
social_relations_min['user_id_int'] = social_relations_min['user_id'].map(user_id_mapping)
social_relations_min["friends_int"] = social_relations_min["friends"].map(user_id_mapping)
# FINAL 2.0
social_relations_min_int = social_relations_min[["user_id_int", "friends_int"]]
checkins_capri_train_test_tune['user_id_int'] = checkins_capri_train_test_tune['user_id'].map(user_id_mapping)
checkins_capri_train_test_tune['business_id_int'] = checkins_capri_train_test_tune['business_id'].map(business_id_mapping)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/606899870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_min['user_id_int'], user_id_map = pd.factorize(checkins_capri_min['user_id'])
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/606899870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_min['business_id_int'], business_id_map = pd.factorize(checkins_capri_min['business_id'])


******

## Next To-Do's: 

Potentially changing the hash's to numeric id's

constructing a train, test and tuning set with all the user'id's present in each set 